In [2]:
from config import *

import gensim

import glob
import os
import re

import numpy as np
import pandas as pd

import json

In [3]:
years = [str(year) for year in range(START_YEAR, END_YEAR + 1)]

In [11]:
THEME = 'GDP'
TOKENIZED_ARTICLES_PATH = TEMP_PATH + '/%s/%s_Articles_Tokenized_%s.json'

all_tok_articles = []
for year in years:
    with open(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year)) as f:
        all_tok_articles.extend(json.load(f))
    print(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year), 'done!')

# Only from 1996 - 2016
print(len(all_tok_articles))

# dictionary_all = gensim.corpora.Dictionary(all_tok_articles)
# print('Dictionary created!')

dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s.dict' % (THEME, THEME))
# corpus_all = [dictionary_all.doc2bow(doc) for doc in all_tok_articles]


class MyCorpus:
    def __iter__(self):
        for doc in all_tok_articles:
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary_all.doc2bow(doc)

corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [13]:
gensim.corpora.MmCorpus.serialize(TEMP_PATH + '/%s/%s.mm' % (THEME, THEME), corpus_memory_friendly)

TypeError: 'MyCorpus' object is not subscriptable

In [12]:
# Storing the dict/ Corpus for future use incase of issues/ notebook crashes
dictionary_all.save(TEMP_PATH + '/%s/%s.dict' % (THEME, THEME))
gensim.corpora.MmCorpus.serialize(TEMP_PATH + '/%s/%s.mm' % (THEME, THEME), corpus_all)

In [ ]:
del all_tok_articles

In [ ]:
# Get Time Slices

In [ ]:
time_slices = pd.read_csv('Summary Stat Tables/%s_Article_Count.csv' % THEME, index_col=0)
time_slices.sort_index(inplace=True)
time_slices.index = pd.to_datetime(time_slices.index)
time_slices.groupby(time_slices.index.year)['No. of GDP Articles'].sum()[:-4]


In [ ]:
time_slices

In [ ]:
# LDA Seq Model

In [37]:
from gensim.models import ldaseqmodel

In [ ]:
time_slices = time_slices.groupby(time_slices.index.year)['No. of Volatility Articles'].sum().values

In [32]:
slices_1996 = time_slices.sort_index()[:12]['No. of GDP Articles'].values

In [35]:
corpus_all = gensim.corpora.MmCorpus(TEMP_PATH + '/%s/%s.mm' % (THEME, THEME))

In [39]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [40]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus_all[:30085], id2word=dictionary_all, time_slice=slices_1996, num_topics=5)

2021-04-25 16:45:16,649 : INFO : using symmetric eta at 0.2
2021-04-25 16:45:16,704 : INFO : using serial LDA version on this node
2021-04-25 16:45:16,937 : INFO : running online (multi-pass) LDA training, 5 topics, 10 passes over the supplied corpus of 30085 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2021-04-25 16:45:17,964 : INFO : PROGRESS: pass 0, at document #2000/30085
2021-04-25 16:45:19,482 : INFO : merging changes from 2000 documents into a model of 30085 documents
2021-04-25 16:45:19,589 : INFO : topic #0 (0.010): 0.009*"jan" + 0.005*"due" + 0.003*"budget" + 0.003*"cents" + 0.003*"department" + 0.003*"united" + 0.003*"dec" + 0.003*"oil" + 0.003*"states" + 0.003*"company"
2021-04-25 16:45:19,593 : INFO : topic #1 (0.010): 0.006*"jan" + 0.004*"budget" + 0.003*"europe" + 0.003*"high" + 0.003*"oil" + 0.003*"united" + 0.003*"per" + 0.003*"time" + 0.002*"march" + 0.002*"cen

KeyboardInterrupt: 

In [10]:
# import sys

# local_vars = list(locals().items())
# total_mem = 0
# for var, obj in local_vars:
#     total_mem += sys.getsizeof(obj)/float(1024*1024)

# total_mem